In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from joblib import load

TEST = 0
TRAIN = 1
SUPP = 2
setMap = ["test", "train", "supp"]

In [ ]:
# ceebs importing
def slide_window(img, window_shape, step, eval_func):
    matches = []
    print(f"\nwindow of size {window_shape[0]} x {window_shape[1]}", flush=True)
    for start_row in tqdm(range(0, img.shape[0], step[1])):
        for start_col in range(0, img.shape[1], step[0]):
            extract = img[start_row : start_row + window_shape[1], start_col : start_col + window_shape[0]]
            try:
                if eval_func(extract, (start_col, start_row)):
                    matches.append((start_col, start_row))
            except:
                print(extract)
                print(window_shape)
                print(img.shape)
                print(start_row, start_row + window_shape[1], start_col, start_col + window_shape[0])
                raise Exception
    print(f"\n{len(matches)} matches")
    return matches

def multi_level_detect(img, min_window, max_window_width, factor, step_ratio, eval_func):
    window = np.array(min_window)
    boxes = []
    while window[0] <= max_window_width:
        step = np.rint(window * step_ratio).astype(int)
        matches = slide_window(img, window, step, eval_func)
        boxes += [(location, *window) for location in matches]
        window = np.rint(window * factor).astype(int)
    return boxes

def cnn_eval(img, loc):
    model = load('model.joblib')
    resized = cv.resize(img, (40, 30))
    return model.predict(np.array([resized]))[0] > 0.5

In [1]:
def convert_boxes_to_vertices(boxes):
    vertices = []
    for box in boxes:
        left = box[0][0]
        right = box[0][0] + box[1]
        top = box[0][1]
        bot = box[0][1] + box[2]
        vertices += [[(left, bot), (left, top), (right, top), (right, bot)]]
    
    return np.array(vertices)
    
def mask_vehicles(img, mask, ignore_mask_color):
    rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    # rohans parameters
    boxes = multi_level_detect(rgb, (80, 60), 400, 1.5, 0.3, cnn_eval)
    vertices = convert_boxes_to_vertices(boxes)
    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    return mask

In [2]:
def Preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.GaussianBlur(gray, (5, 5), 0)

def OtsuImg(img):
    ret, otsu = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return otsu

def transformThreshold(img, thresholdValue):
    outputImage = np.full(img.shape, 0)

    for i in range(outputImage.shape[0]):
        for j in range(outputImage.shape[1]):
            outputImage[i,j] = 255 * (img[i,j] > thresholdValue)
    
    return np.uint8(outputImage)


def Canny(img):
    return cv2.Canny(img, 100, 200)

def RegionOfInterest(img):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """

    lowerLeftPoint = [0, 720]
    lowerRightPoint = [1280, 720]
    upperLeftPoint = [120, 360]
    upperRightPoint = [1100, 360]

    vertices = np.array([[lowerLeftPoint, upperLeftPoint, upperRightPoint, lowerRightPoint]], dtype=np.int32)

    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with 
    #depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    masked_image = maskVehicles(img, mask, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    try:
        if lines.any():
            for line in lines:
                if line.any():
                    for x1,y1,x2,y2 in line:
                        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    except:
        pass
    

def hough_lines(img):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    rho = 1
    theta = np.pi/180
    threshold = 30
    min_line_len = 20 
    max_line_gap = 20

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]))
    
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    
    draw_lines(line_img, lines)
    return line_img

In [3]:
def getImage(n, set):
    path = "benchmark_velocity_supp/supp_img/" + str(n).zfill(4) + ".jpg"
    if (set == TEST or set == TRAIN):
        path = "benchmark_velocity_" + setMap[set] + "/clips/" + str(n) + "/imgs/040.jpg"
    
    print(path)
    img = cv2.imread(path)
    return img

In [ ]:
for i in range(1,100):
    print(i)
    img = getImage(i, TRAIN)
    img = RegionOfInterest(img)
    preprocessedImg = Preprocess(img)
    thresholdImg = transformThreshold(preprocessedImg, 140)
    thresholdImg2 = OtsuImg(preprocessedImg)

    cannyImg = Canny(np.uint8(thresholdImg))
    maskedImg = RegionOfInterest(cannyImg)
    houged = hough_lines(maskedImg)

    if (not os.path.exists('output')):
        print('creating output dir')
        os.mkdir('./output')

    cv2.imwrite("output/output" + str(i) + ".jpg", np.uint8(RegionOfInterest(thresholdImg)))
    cv2.imwrite("output/output" + str(i) + "Original.jpg", preprocessedImg)



In [ ]:
def evaluate():
    